# Restaurant Reviews NLP implementation
This is my first project on DataBricks AWS using PySpark. This is my first NLP problem in ML.

## Objective
Categorize customer sentiment using the provided review. The provided dataset has

# Familiarizing
There are several practices in NLP that need just a bit of exploration. In order:
* Tokenization
  * Splitting of sentences into individual words for further processing, seems to be one of the first steps in NLP data preparation.
* Stop-Word Removal
  * Removal of unnecessary words which do not provide any contextual meaning to the sentences. Words like 'the', 'of', 'to', etc.
* N-Grams
  * Grouping of individual words into n-long 'sentences'. This can be useful for identifying particular n-grams which can provide important information. Things such as 'tasted bad', 'great price', etc.
* Term Frequency
  * Checking the frequency of particular terms, which may or may not have correlation with outputs. Inverse frequency is also commonly used under the following principle: Words that appear exceedingly often most likely have less unique actionable information than less-frequent words. For exmaple, comparing 'stellar' with 'okay'.

In [ ]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, StructField, StructType, BooleanType, ArrayType, TimestampType

In [ ]:
df = sqlContext.sql("Select * FROM nlp_restaurant_reviews")
df.show(5)

+--------------------+-----+
 Review|Liked|
+--------------------+-----+
Wow... Loved this...| 1|
 Crust is not good.| 0|
Not tasty and the...| 0|
Stopped by during...| 1|
The selection on ...| 1|
+--------------------+-----+
only showing top 5 rows

In [ ]:
df.printSchema()

root
-- Review: string (nullable = true)
-- Liked: integer (nullable = true)

In [ ]:
df.count()

Out[4]: 1000

## Tokenization
Splitting of sentences into individual words for processing

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import udf

In [ ]:
tokenizer = Tokenizer(inputCol='Review', outputCol='words') # Splits sentences into whole words

regexTokenizer = RegexTokenizer(inputCol='Review', outputCol='words', pattern='\\W') # Splits sentences into whole words, ignores punctuation

countTokens = udf(lambda p: len(p), IntegerType()) # udf applies the lambda function to each row when called on a column

In [ ]:
tokenized = tokenizer.transform(df)
tokenized.withColumn('tokens', countTokens(col('words'))).show()

+--------------------+-----+--------------------+------+
 Review|Liked| words|tokens|
+--------------------+-----+--------------------+------+
Wow... Loved this...| 1|[wow..., loved, t...| 4|
 Crust is not good.| 0|[crust, is, not, ...| 4|
Not tasty and the...| 0|[not, tasty, and,...| 8|
Stopped by during...| 1|[stopped, by, dur...| 15|
The selection on ...| 1|[the, selection, ...| 12|
Now I am getting ...| 0|[now, i, am, gett...| 11|
Honeslty it didn'...| 0|[honeslty, it, di...| 6|
The potatoes were...| 0|[the, potatoes, w...| 22|
The fries were gr...| 1|[the, fries, were...| 5|
 A great touch.| 1| [a, great, touch.]| 3|
Service was very ...| 1|[service, was, ve...| 4|
 Would not go back.| 0|[would, not, go, ...| 4|
The cashier had n...| 0|[the, cashier, ha...| 21|
I tried the Cape ...| 1|[i, tried, the, c...| 9|
I was disgusted b...| 0|[i, was, disguste...| 12|
I was shocked bec...| 0|[i, was, shocked,...| 9|
 Highly recommended.| 1|[highly, recommen...| 2|
Waitress was a li...| 0|[waitress, was, a...| 7|
This place is not...| 0|[this, place, is,...| 10|
did not like at all.| 0|[did, not, like, ...| 5|
+--------------------+-----+--------------------+------+
only showing top 20 rows

## Stop Word Removal

Removal of common words which add little to no meaning to the sentence

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
df.show(5, truncate=False)

+---------------------------------------------------------------------------------------+-----+
Review |Liked|
+---------------------------------------------------------------------------------------+-----+
Wow... Loved this place. |1 |
Crust is not good. |0 |
Not tasty and the texture was just nasty. |0 |
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.|1 |
The selection on the menu was great and so were the prices. |1 |
+---------------------------------------------------------------------------------------+-----+
only showing top 5 rows

In [ ]:
remover = StopWordsRemover(inputCol='words', outputCol='cleaned')
remover.transform(tokenized).select('cleaned').show(5, truncate=False)

+----------------------------------------------------------------------------+
cleaned |
+----------------------------------------------------------------------------+
[wow..., loved, place.] |
[crust, good.] |
[tasty, texture, nasty.] |
[stopped, late, may, bank, holiday, rick, steve, recommendation, loved, it.]|
[selection, menu, great, prices.] |
+----------------------------------------------------------------------------+
only showing top 5 rows

## n-grams

In [ ]:
from pyspark.ml.feature import NGram

In [ ]:
bigram = NGram(n=2, inputCol='words', outputCol='bigrams')
bigram_df = bigram.transform(tokenized).select('bigrams').show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
bigrams |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[wow... loved, loved this, this place.] |
[crust is, is not, not good.] |
[not tasty, tasty and, and the, the texture, texture was, was just, just nasty.] |
[stopped by, by during, during the, the late, late may, may bank, bank holiday, holiday off, off rick, rick steve, steve recommendation, recommendation and, and loved, loved it.]|
[the selection, selection on, on the, the menu, menu was, was great, great and, and so, so were, were the, the prices.] |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

## Term Freq; Inverse Doc Freq (TF-IDF)
Importance of specific words. Generally, the more a word is used, the less unique meaning it carries within a corpus. Thus, inverse frequency is used.

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [ ]:
tokenizer = RegexTokenizer(inputCol='Review', outputCol='words', pattern='\\W')
words = tokenizer.transform(df)
words.show(5, truncate=False)

+---------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------+
Review |Liked|words |
+---------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------+
Wow... Loved this place. |1 |[wow, loved, this, place] |
Crust is not good. |0 |[crust, is, not, good] |
Not tasty and the texture was just nasty. |0 |[not, tasty, and, the, texture, was, just, nasty] |
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.|1 |[stopped, by, during, the, late, may, bank, holiday, off, rick, steve, recommendation, and, loved, it]|
The selection on the menu was great and so were the prices. |1 |[the, selection, on, the, menu, was, great, and, so, were, the, prices] |
+---------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
hashingTF = HashingTF(inputCol='words', outputCol='rawFeatures', numFeatures=20)
featurized = hashingTF.transform(words)
featurized.show(5)

+--------------------+-----+--------------------+--------------------+
 Review|Liked| words| rawFeatures|
+--------------------+-----+--------------------+--------------------+
Wow... Loved this...| 1|[wow, loved, this...|(20,[9,10,13,19],...|
 Crust is not good.| 0|[crust, is, not, ...|(20,[5,8,9,18],[1...|
Not tasty and the...| 0|[not, tasty, and,...|(20,[2,3,4,5,7,11...|
Stopped by during...| 1|[stopped, by, dur...|(20,[0,1,2,3,4,6,...|
The selection on ...| 1|[the, selection, ...|(20,[2,3,10,11,16...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [ ]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized)
rescale = idf_model.transform(featurized)
rescale.select('Liked', 'features').show(5)

+-----+--------------------+
Liked| features|
+-----+--------------------+
 1|(20,[9,10,13,19],...|
 0|(20,[5,8,9,18],[1...|
 0|(20,[2,3,4,5,7,11...|
 1|(20,[0,1,2,3,4,6,...|
 1|(20,[2,3,10,11,16...|
+-----+--------------------+
only showing top 5 rows

# Prototype Model
From what I understand, NLP follows the following 'simple' procedure:
1) Process text data
2) Make features numeric (i.e. vectorization)
3) Train using existing ML models (such as random forest classifier)

In my research the 'sparknlp' library provides access to more refined tools for NLP tasks, and so now I will be working with that. My original dataframe from above is still intact, I will be starting there.

In [ ]:
import sparknlp
from sparknlp.base import * # Change once all used modules are known
from sparknlp.annotator import * # Change once all used modules are known
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier, LinearSVC, LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

In [ ]:
spark = sparknlp.start()

## Annotation

The text needs to be formatted and cleaned as much as possible prior to the vectorization of the data. The steps I wish to implement, to some degree, are as follows. For all of these points, I will modify my approach as new information becomes available to me.
1) **Tokenization**
    * Splitting of reviews into individual words. This may or may not involve multiple steps, a custom RegEx, or more.
2) **Spell-checking**
    * All words should be spell-checked prior to any future steps for obvious reasons.
3) **Stop-words**
    * Removal of unnecessary words that do not contribute significant meaning to the sentence. Removal of these words places more emphasis on the words that matter during training.
4) **Lemmatization**
    * Words should be reduced to their lemma forms in order to make the text more readable to the computer (i.e. Doing, Does, Did -> Do). In theory, this should strengthen the patterns between certain words and their corresponding meaning as interpreted by the algorithm.

In [ ]:
document = DocumentAssembler()\
                .setInputCol('Review')\
                .setOutputCol('document')\

sentence = SentenceDetector()\
                .setInputCols('document')\
                .setOutputCol('sentence')

tokenizer = Tokenizer()\
                .setInputCols('sentence')\
                .setOutputCol('token')

checker = NorvigSweetingModel()\
                .pretrained()\
                .setInputCols('token')\
                .setOutputCol('checked')

stopwords = StopWordsCleaner()\
                .pretrained('stopwords_en', 'en')\
                .setInputCols('checked')\
                .setOutputCol('cleaned')

lemmatizer = LemmatizerModel()\
                .pretrained()\
                .setInputCols('cleaned')\
                .setOutputCol('lemma')

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]
stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[ | ][OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]

In [ ]:
pipeline = Pipeline().setStages([document, sentence, tokenizer, checker, stopwords, lemmatizer])
model = pipeline.fit(df)
result = model.transform(df)

In [ ]:
result.select('Review', 'lemma.result').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+
Review |result |
+---------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+
Wow... Loved this place. |[Wow, ..., move, place, .] |
Crust is not good. |[Crust, good, .] |
Not tasty and the texture was just nasty. |[tasty, texture, nasty, .] |
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it. |[Stopped, late, bank, holiday, Rick, Steve, recommendation, love, .]|
The selection on the menu was great and so were the prices. |[selection, menu, great, price, .] |
Now I am getting angry and I want my damn pho. |[angry, damn, pho, .] |
Honeslty it didn't taste THAT fresh.) |[Honestly, taste, fresh, ., )] |
The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer.|[potato, rubber, make, ahead, time, warm, .] |
The fries were great too. |[fry, great, .] |
A great touch. |[great, touch, .] |
Service was very prompt. |[Service, prompt, .] |
Would not go back. |[back, .] |
The cashier had no care what so ever on what I had to say it still ended up being wayyy overpriced. |[cashier, care, end, overpriced, .] |
I tried the Cape Cod ravoli, chicken, with cranberry...mmmm! |[Cape, Cod, ravioli, ,, chicken, ,, cranberry, ., ., ., mmm, !] |
I was disgusted because I was pretty sure that was human hair. |[disgust, pretty, human, hair, .] |
I was shocked because no signs indicate cash only. |[shock, sign, cash, .] |
Highly recommended. |[Highly, recommend, .] |
Waitress was a little slow in service. |[waitress, slow, service, .] |
This place is not worth your time, let alone Vegas. |[place, worth, time, ,, Vegas, .] |
did not like at all. |[.] |
+---------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+
only showing top 20 rows

There is an issue with the first line spell checking 'Loved' to 'moved' and eventually lemmatized to 'move'. Perhaps a different tokenizer without punctuation would help. Important negators like 'not' are being flagged by the stopwords filter, and this is a big problem. A review like 'Crust is not good' being turned into 'Crust good' is clearly a flipping of the sentiment of the review. I will need to look into this.

I want to create a custom RegEx that will exclude almost all punctuation except for apostrophes and hyphens. As I have zero experience with RegEx, I found a website that allows me to test different RegEx expressions and see how they affect sample text. Using this, I was able to create the RegEx expression r"[a-zA-Z0-9\'\-]" which should include everything I need, and exclude the rest. Since this is being applied after a sentencer, I do not need to worry about single periods.

In [ ]:
tokenizer = RegexTokenizer()\
                .setInputCols('sentence')\
                .setOutputCol('token')\
                .setPattern(r'[^a-zA-Z0-9_\']')

checker = ContextSpellCheckerModel\
                .pretrained()\
                .setInputCols("token")\
                .setOutputCol("checked")

lemmatizer = LemmatizerModel()\
                .pretrained()\
                .setInputCols('checked')\
                .setOutputCol('lemma')

stopwords = StopWordsCleaner()\
                .pretrained('stopwords_iso', 'en')\
                .setInputCols('lemma')\
                .setOutputCol('cleaned')

finisher = Finisher()\
                .setInputCols('lemma')\
                .setOutputCols('result')

spellcheck_dl download started this may take some time.
Approximate size to download 95.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]
stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[ | ][OK!]

In [ ]:
pipeline = Pipeline().setStages([document, sentence, tokenizer, checker, lemmatizer, finisher])
model = pipeline.fit(df)
result = model.transform(df)

In [ ]:
result.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
Review |Liked|finished |cleaned |rawFeatures |features |label|
+---------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
Wow... Loved this place. |1 |[Wow, Loved, this, place] |[Wow, Loved, place] |(1935,[10,17,810,839],[1.0,1.0,1.0,1.0]) |(1935,[10,17,810,839],[2.226623552191001,2.4089451089849554,5.810142490647111,5.810142490647111]) |1 |
Crust is not good. |0 |[Crust, be, not, good] |[Crust, not, good] |(1935,[0,11,14,1239],[1.0,1.0,1.0,1.0]) |(1935,[0,11,14,1239],[0.5754751511755302,2.235925944853314,2.2837819660309497,6.215607598755275]) |0 |
Not tasty and the texture was just nasty. |0 |[Not, tasty, and, the, texture, be, just, nasty] |[Not, tasty, texture, just, nasty] |(1935,[0,1,2,43,156,171,482,875],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |(1935,[0,1,2,43,156,171,482,875],[0.5754751511755302,1.1066364039381575,1.1096621248546947,3.443018876515494,4.4238481295272205,4.51085950651685,5.52246041819533,5.810142490647111]) |0 |
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it. |1 |[Stopped, by, during, the, late, May, bank, holiday, off, Rick, Steve, recommendation, and, love, it] |[Stopped, late, May, bank, Rick, Steve, recommendation, love] |(1935,[1,2,12,58,93,180,237,344,559,702,981,1130,1441,1917,1918],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |(1935,[1,2,12,58,93,180,237,344,559,702,981,1130,1441,1917,1918],[1.1066364039381575,1.1096621248546947,2.3237873006446486,3.6506582412937387,3.96431580014878,4.6061696863211745,4.829313237635384,5.116995310087166,5.52246041819533,5.810142490647111,6.215607598755275,6.215607598755275,6.215607598755275,6.215607598755275,6.215607598755275]) |1 |
The selection on the menu was great and so were the prices. |1 |[The, selection, on, the, menu, be, great, and, so, be, the, price] |[selection, menu, great, price] |(1935,[0,1,2,6,27,32,34,84,106,144],[2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |(1935,[0,1,2,6,27,32,34,84,106,144],[1.1509503023510603,2.213272807876315,1.1096621248546947,1.7213689734744655,2.831217335409501,3.0375537684073297,3.016934481204594,3.8642323415917974,4.075541435259004,4.4238481295272205]) |1 |
Now I am getting angry and I want my damn pho. |0 |

A new spell checker, as well as a custom RegEx fixed my issues so far. There are still some errors in the first few rows (i.e. ravoli -> revolt; overpriced -> override), but these are hard to fix and I have to accept some level of error, especially on my first ever NLP project. I still want to explore using a stopword filter, but I'm not quite sure at this point how to ensure I keep contextual descriptors like 'not'. I've tried both pretrained stop-word models available in the spark-nlp library. These are build on the pyspark.ml.features.StopWordsRemover, so that wouldn't work either. My last, very rudimentary option, is to do a custom stop-word list.

I'm simply going to look through the first 25 rows and pick out any words I don't believe are necessary. Later on I may use a count vectorizer to find the terms with the most frequency, however this will take a long computation time and for now, I just want to get a prototype working.

## Vectorization

The text now needs to be given a numeric format for usage by the ML classifier. To my knowledge, there are two paradigms in which this can be done; text-frequency based, and pretrained-embeddings based. For this first project, I will be using text-frequency based vectorization, and I will be using a count vectorizer followed by an IDF.

In [ ]:
stop_words = ['the', 'be', 'now', 'get', 'on', 'and', 'so', 'want', 'I', 'it', 'that', 'honesty', 'you', 'they', 'them', 'this', 'by', 'during', 'holiday', 'off', 'back', 'what',
             'say', 'to', 'still', 'end', 'up', 'way', 'little', 'in', 'let', 'alone', 'at', 'all', 'because', 'oh', 'stuff', 'red', 'that\'s', 'a', 'of', 'some']

remover = StopWordsRemover(stopWords = stop_words)\
            .setInputCol('finished')\
            .setOutputCol('cleaned')

finisher = Finisher()\
                .setInputCols('lemma')\
                .setOutputCols('finished')

countVec = CountVectorizer()\
            .setInputCol('finished')\
            .setOutputCol('rawFeatures')

idf = IDF()\
            .setInputCol('rawFeatures')\
            .setOutputCol('features')

pipeline = Pipeline().setStages([document, sentence, tokenizer, checker, lemmatizer, finisher, remover, countVec, idf])
model = pipeline.fit(df)
result = model.transform(df)

result.show()

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
 Review|Liked| finished| cleaned| rawFeatures| features|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
Wow... Loved this...| 1|[Wow, Loved, this...| [Wow, Loved, place]|(1935,[10,17,810,...|(1935,[10,17,810,...|
 Crust is not good.| 0|[Crust, be, not, ...| [Crust, not, good]|(1935,[0,11,14,12...|(1935,[0,11,14,12...|
Not tasty and the...| 0|[Not, tasty, and,...|[Not, tasty, text...|(1935,[0,1,2,43,1...|(1935,[0,1,2,43,1...|
Stopped by during...| 1|[Stopped, by, dur...|[Stopped, late, M...|(1935,[1,2,12,58,...|(1935,[1,2,12,58,...|
The selection on ...| 1|[The, selection, ...|[selection, menu,...|(1935,[0,1,2,6,27...|(1935,[0,1,2,6,27...|
Now I am getting ...| 0|[Now, I, be, get,...| [angry, damn, Rho]|(1935,[0,2,3,18,3...|(1935,[0,2,3,18,3...|
Honeslty it didn'...| 0|[Honesty, it, did...|[didn't, taste, f...|(1935,[12,77,126,...|(1935,[12,77,126,...|
The potatoes were...| 0|[The, potato, be,...|[potato, like, ru...|(1935,[0,2,4,6,7,...|(1935,[0,2,4,6,7,...|
The fries were gr...| 1|[The, fry, be, gr...| [fry, great, too]|(1935,[0,6,32,109...|(1935,[0,6,32,109...|
 A great touch.| 1| [A, great, touch]| [great, touch]|(1935,[32,190,570...|(1935,[32,190,570...|
Service was very ...| 1|[Service, be, ver...|[Service, very, p...|(1935,[0,25,115,1...|(1935,[0,25,115,1...|
 Would not go back.| 0|[Would, not, go, ...| [Would, not, go]|(1935,[14,21,26,4...|(1935,[14,21,26,4...|
The cashier had n...| 0|[The, Fasher, hav...|[Fasher, have, no...|(1935,[0,3,5,6,7,...|(1935,[0,3,5,6,7,...|
I tried the Cape ...| 1|[I, try, the, Cap...|[try, Cape, Cod, ...|(1935,[1,3,22,112...|(1935,[1,3,22,112...|
I was disgusted b...| 0|[I, be, disgust, ...|[disgust, pretty,...|(1935,[0,3,24,97,...|(1935,[0,3,24,97,...|
I was shocked bec...| 0|[I, be, shock, be...|[shock, no, sign,...|(1935,[0,3,59,82,...|(1935,[0,3,59,82,...|
 Highly recommended.| 1| [Highly, recommend]| [Highly, recommend]|(1935,[118,589],[...|(1935,[118,589],[...|
Waitress was a li...| 0|[Waitress, be, a,...|[Waitress, slow, ...|(1935,[0,4,15,23,...|(1935,[0,4,15,23,...|
This place is not...| 0|[This, place, be,...|[place, not, wort...|(1935,[0,10,14,19...|(1935,[0,10,14,19...|
did not like at all.| 0|[do, not, like, a...| [do, not, like]|(1935,[14,31,33,3...|(1935,[14,31,33,3...|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

On second thought, once the model is trained, it would in-fact be useful to have a CountVectorization map rather than a hashmap. The ability to distinguish which buzz-words are associated with positive and negative reviews is quite useful, and it is cumbersome to do this with a hashed count. It only takes around 4-5 minutes to run on this small dataset, so it's not bad at all, actually. I really want to get some predictions going.

## Model training and evaluation

Finally I can train a model on this very rough pipeline. I simply want to get a prototype working so that I can understand all I have until now. Afterwards, I will set up a proper full-length pipeline with several models to try and hyperparameter tuning. For now, I will train on a LogisticRegression classifier (weird name)

In [ ]:
model_data = result.withColumn('label', col('Liked')).select(['features', 'label'])
model_data.show(5)

train, test = model_data.randomSplit([0.9, 0.1], seed=31415)
train.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
(1935,[10,17,810,...| 1|
(1935,[0,11,14,12...| 0|
(1935,[0,1,2,43,1...| 0|
(1935,[1,2,12,58,...| 1|
(1935,[0,1,2,6,27...| 1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
 features|label|
+--------------------+-----+
(1935,[0,1,2,3,4,...| 1|
(1935,[0,1,2,3,4,...| 1|
(1935,[0,1,2,3,4,...| 1|
(1935,[0,1,2,3,4,...| 0|
(1935,[0,1,2,3,4,...| 1|
(1935,[0,1,2,3,5,...| 0|
(1935,[0,1,2,3,5,...| 0|
(1935,[0,1,2,3,5,...| 0|
(1935,[0,1,2,3,6,...| 0|
(1935,[0,1,2,3,6,...| 0|
(1935,[0,1,2,3,6,...| 1|
(1935,[0,1,2,3,7,...| 1|
(1935,[0,1,2,3,7,...| 1|
(1935,[0,1,2,3,7,...| 0|
(1935,[0,1,2,3,7,...| 1|
(1935,[0,1,2,3,7,...| 1|
(1935,[0,1,2,3,8,...| 0|
(1935,[0,1,2,3,8,...| 1|
(1935,[0,1,2,3,8,...| 0|
(1935,[0,1,2,3,9,...| 1|
+--------------------+-----+
only showing top 20 rows

In [ ]:
lr = LogisticRegression()
lrModel = lr.fit(train)

In [ ]:
lr_pred = lrModel.evaluate(test)

In [ ]:
lr_pred.accuracy

Out[91]: 0.7659574468085106

# Putting it all together

Now I can start looking at training entire pipelined models, applying grid searches on parameters, and testing different models. I will now describe a high-level summary of the tasks to be completed:

1) Pipeline for initial data processing, including the following steps:
    * Document, Sentencing -- Standard
    * Tokenization -- Standard
    * Spell checking -- ContextSpellChecker provided the best results in the exploration phase above
    * Lemmatization -- Standard
    * Stop-word removal -- Need to test several approaches:
        * No stop-words
        * Custom list as used above
        * Pretrained model
    * Finisher (Standard)
    * HashingTF (For speed vs CountVectorizer)
    * IDF Layer (Standard)
    
2) Model pipelines constructed where previous pipeline ends:
    * One for each model to be tested
    * Use cross validation and parameter grids to identify strongest candidates
    
The model pipelines will be separate from the data processing pipeline due to the fact that the data processing pipeline will be identical regardless of the model used. This will cut down on compute time at the cost of memory. Currently, memory is not an issue.

In [ ]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import RegexTokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel, TrainValidationSplit
from pyspark.sql.functions import col

import sparknlp
from sparknlp.annotator import * # Change once all used modules are known
from sparknlp.base import * # Change once all used modules are known

In [ ]:
# ---=== Base pipeline with pretrained stopwords ===---

document = DocumentAssembler()\
                .setInputCol('Review')\
                .setOutputCol('document')\

sentence = SentenceDetector()\
                .setInputCols('document')\
                .setOutputCol('sentence')

tokenizer = RegexTokenizer()\
                .setInputCols('sentence')\
                .setOutputCol('token')\
                .setPattern(r'[^a-zA-Z0-9_\']')

checker = ContextSpellCheckerModel\
                .pretrained()\
                .setInputCols("token")\
                .setOutputCol("checked")

lemmatizer = LemmatizerModel()\
                .pretrained()\
                .setInputCols('checked')\
                .setOutputCol('lemma')

stopwords = StopWordsCleaner()\
                .pretrained('stopwords_iso', 'en')\
                .setInputCols('lemma')\
                .setOutputCol('cleaned')

finisher = Finisher()\
                .setInputCols('cleaned')\
                .setOutputCols('finished')

hashingTF = HashingTF()\
                .setInputCol('finished')\
                .setOutputCol('rawFeatures')

idf = IDF()\
                .setInputCol('rawFeatures')\
                .setOutputCol('features')

pipe_sw_pre = Pipeline()\
                .setStages([document, sentence, tokenizer, checker, 
                            lemmatizer, stopwords, finisher, hashingTF, idf])

# ---=== Custom stop words in the pipeline ===---

stop_words = ['the', 'be', 'now', 'get', 'on', 'and', 'so', 'want', 'I', 'it', 'that', 'honesty', 'you', 'they', 'them', 'this', 'by', 'during', 'holiday', 'off', 'back', 'what',
             'say', 'to', 'still', 'end', 'up', 'way', 'little', 'in', 'let', 'alone', 'at', 'all', 'because', 'oh', 'stuff', 'red', 'that\'s', 'a', 'of', 'some']

finisher.setInputCols('lemma')\
        .setOutputCols('finished')

stopwords = StopWordsRemover(stopWords = stop_words)\
                .setInputCol('finished')\
                .setOutputCol('cleaned')

hashingTF.setInputCol('cleaned')\
         .setOutputCol('rawFeatures')
                

pipe_sw_cstm = Pipeline()\
                .setStages([document, sentence, tokenizer, checker, 
                            lemmatizer, finisher, stopwords, hashingTF, idf])

# ---=== No stop words in the pipeline ===---

finisher.setInputCols('lemma')\
        .setOutputCols('finished')

hashingTF.setInputCol('finished')\
         .setOutputCol('rawFeatures')

pipe_sw_none = Pipeline()\
                .setStages([document, sentence, tokenizer, checker, 
                            lemmatizer, finisher, hashingTF, idf])

spellcheck_dl download started this may take some time.
Approximate size to download 95.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]
stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[ | ][OK!]

I wish to use a classification model that allows for explainability of the data. This is a supervised classification problem, and so the possible models I am aware of that meet these criteria are Decision Trees and Logistic Regression. I will start with random-ish values for the parameters and adjust as results come in.

In [ ]:
lr = LogisticRegression()
lr_params = ParamGridBuilder()\
            .addGrid(lr.regParam, [0.1, 0.01])\
            .addGrid(lr.maxIter, [1, 5, 10, 25])\
            .build()

dt = DecisionTreeClassifier()
dt_params = ParamGridBuilder()\
            .addGrid(dt.maxBins, [10,50,100])\
            .addGrid(dt.maxDepth, [10, 50, 100])\
            .build()

In [ ]:
df = sqlContext.sql("Select * FROM nlp_restaurant_reviews")
df = df.withColumn('label', col('Liked')).select(['label', 'Review'])
train, test = df.randomSplit([0.8, 0.2], seed = 31415)

model_info = {
    'lr' : lr,
    'dt' : dt
}

param_info = {
    'lr' : lr_params,
    'dt' : dt_params
}

pipe_info = {
    'sw_pre' : pipe_sw_pre,
    'sw_cstm' : pipe_sw_cstm,
    'sw_none' : pipe_sw_none
}



# , pipe_sw_custom, pipe_sw_none
# , 'dt'

# I will be constructing a JSON-like list containing the results of the calculations.
        
"""
cv = TrainValidationSplit(estimator = pipe_added,
                   estimatorParamMaps = lr_params,
                   evaluator = BinaryClassificationEvaluator(),
                   seed = 31415,
                   parallelism=1
)

cvModel = cv.fit(train)
pred = cvModel.transform(test)
"""

Out[10]: '\ncv = TrainValidationSplit(estimator = pipe_added,\n estimatorParamMaps = lr_params,\n evaluator = BinaryClassificationEvaluator(),\n seed = 31415,\n parallelism=1\n)\n\ncvModel = cv.fit(train)\npred = cvModel.transform(test)\n'

I wanted to use the CrossValidator object to test different parameter distributions as well as the separate models, however, I was running into an error with no documentation online. I've posted a StackOverflow request but to no avail. For now, I will forego the hyperparameter tuning by use of CrossValidator (TrainValidationSplit raised same error) and simply test the different pipelines with default parameters.

In [ ]:
results_info = {}
evaluator = BinaryClassificationEvaluator()
df = sqlContext.sql("Select * FROM nlp_restaurant_reviews")
df = df.withColumn('label', col('Liked')).select(['label', 'Review'])

for pipe_label, pipe in pipe_info.items():
    model_results = {}

    for model_label, model in model_info.items():
        train, test = df.randomSplit([0.8, 0.2], seed = 31415)
        
        pipe_added = Pipeline().setStages([pipe, model])
        pipe_trained = pipe_added.fit(train)
        
        pipe_data = pipe_trained.transform(test)
        areaUnderROC = evaluator.evaluate(pipe_data)
        
        print(f'{pipe_label} - {model_label}: {areaUnderROC}')
        model_results[model_label] = areaUnderROC
        
        del pipe_data
        
    results_info[pipe_label] = model_results

sw_pre - lr: 0.8132022471910118
sw_pre - dt: 0.4323501872659177
sw_cstm - lr: 0.8132022471910118
sw_cstm - dt: 0.4323501872659177
sw_none - lr: 0.8132022471910118
sw_none - dt: 0.4323501872659177

There is clearly something else going on here, it is extremely unlikely that all three pipelines perform exactly the same. Either my pipelines are being instanced to the same memory locations, or there is a leak in my loop. Regardless, it would seem that the LogisticRegression classification works significantly better than the Decision Tree. Since this is a binary classification, if the accuracy of the Decision Tree is <0.5 I can simply take the complement to achieve a true prediction accuracy >0.5. The above metric however is the area under the ROC curve, and so I should look at the accuracy specifically if I want to apply that reasoning.